In [3]:
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso as lasso
from sklearn.linear_model import LassoCV as lassoCV
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression as lr
from sklearn.preprocessing import normalize as normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr as corr
import math
%matplotlib inline

In [4]:
data = pd.ExcelFile('data_cellline1.xlsx')

In [5]:
#data cleanup
ic50 = data.parse(0).transpose()

rna = data.parse(1).transpose()[1:]
cnv = data.parse(2).transpose()[1:]
mutation = data.parse(3).transpose()[1:]
ic50 = ic50.set_index(0)
train_list = ic50[2]!= '?'
test_list = ic50[2]== '?'

In [6]:
rna_train = rna[train_list]
rna_test = rna[~train_list]
cnv_train = cnv[train_list]
mutation_train = mutation[train_list]
ic50_train = ic50[train_list][[2, 3, 4]]
ic50_test = ic50[~train_list][[2, 3, 4]]

In [5]:
#try how scikit-learn lasso performs on the dataset
#assume that the genes that affect the ic50 response after 24, 40, 72 hours will be roughly the same
#one way to realize this is to do the cross validate between 24, 40, 72 group for paramter selection
#the predictability of the response depreciate over time.
#why not use the first two responses to predict the third
#only used dataset 'rna' based on the results of Prof. Yu
# ls24 = lr()
# ls24.alpha = 10
# ls24.fit(rna_train, ic50_train[2])
# print mse(ls24.predict(rna_train), ic50_train[2])

In [6]:
#check the correlation between the three responses
print corr(ic50_train[2], ic50_train[3])
print corr(ic50_train[2], ic50_train[4])
print corr(ic50_train[3], ic50_train[4])
#the crucial task is to get a good predicter of ic50-24, which itself is a good predicter of ic50-40 and ic50-72

(0.53628491307287618, 5.7506838202445851e-11)
(0.42407962657898152, 5.4950735453706101e-07)
(0.63886296277099752, 3.7553742971781563e-16)


In [7]:
#define a cross validation function to find the best lasso parameter and the best set of parameters.
#What defines best? Try several lasso parameters and select the one that gives the minimum mse
#Use scikit-learn lassoCV as an example
lassocv24 = lassoCV(alphas= np.arange(20, 30, 0.02))
lassocv24.fit(rna_train, ic50_train[2])
#lassocv for response after 24 hours
# Method replaced by multilassoCV

In [ ]:
#One time lasso, not very satisfying
# print lassocv24.alpha_ #finalized alpha by crossvalidation
# print lassocv24.predict(rna_train)
# # print rna_train.apply(lassocv24.predict)
# mselassocv24 = mse(lassocv24.predict(rna_train), ic50_train[2])
# print mselassocv24
# print lassocv24.mse_path_
# print lassocv24.alphas_
# best lasso parameter might be 25.6927604662 at the first run
# try to focus around 20-30 and run again
# the result is 26.24
# best lasso parameter chosen as 26 in the first round selection

In [ ]:
# see the selected parameters
# selected parameters: 7595, 8995, 9415, 14674
# we got a quite small number of features.



# parameters_round1 = []
# for _ in range(18875):
#     if lassocv24.coef_[_]!= 0:
#         parameters_round1.append(_)
# print parameters_round1

In [ ]:
# next step is to use the method of simulated-annealing, add some parameters back and see if we can get better results.
# the next step might be to add/delete genes from the same gene set, see if we can get a better result.


# feature_set = rna_train[parameters_round1]
# for _ in parameters_round1:
#     for __ in parameters_round1:
#         if _ != __:
#             print _, __, 'correlation/ p-valuse is :', corr(feature_set[_],feature_set[__])



# 7595 8995 correlation/ p-valuse is : (0.33558283328581212, 0.00010117831558265309)
# 7595 9415 correlation/ p-valuse is : (0.32752270579913823, 0.00015138509748622306)

# 7595 seems to have some correlation with the other two

In [124]:
# try linear model with 4 selected parameters
# not very satisfying
linear_1stround_4parameters = lr()
linear_1stround_4parameters.fit(feature_set[:109], ic50_train[2][:109])
linear_1stround_4parameters_mse = mse(linear_1stround_4parameters.predict(feature_set[109:]), ic50_train[2][109:])
print linear_1stround_4parameters_mse

NameError: name 'feature_set' is not defined

In [ ]:
linear_2ndround_add1parameter = lr()
for _ in range(18875):
    if _ not in parameters_round1:
        new_parameters = parameters_round1 + [_]
        new_feature_set = rna_train[new_parameters]
        linear_2ndround_add1parameter.fit(new_feature_set[:109], ic50_train[2][:109])
        linear_2ndround_add1parameter_mse = mse(linear_2ndround_add1parameter.predict(new_feature_set[109:]),ic50_train[2][109:])
        if linear_2ndround_add1parameter_mse < linear_1stround_4parameters_mse:
            print 'a better mse of', linear_2ndround_add1parameter_mse, 'with parameters set', new_parameters
            linear_1stround_4parameters_mse = linear_2ndround_add1parameter_mse

In [7]:
# selected 1304 as the new feature to enter the set, run another round
# add a few more features so that we can run another lasso regression
# we have to define a function for this
def add_new_parameters(x, y, loc_param, startingMSE, split_ratio = 0.8, maxiter = 1):
    simple_lr = lr()
    training_set_result = []
    i = 0
    while i <maxiter:
        i = i + 1
        for _ in list(x):
            resultMSE = msetest(x, y, loc_param)
            if _ not in loc_param and np.random.random() < 1:
                if msetest(x, y ,loc_param + [_]) < resultMSE:
                    loc_param = loc_param + [_]
    #     return 'a better MSE of', resultMSE, 'with parameters set', loc_param
        print i
    return loc_param
#                 temp_train_set = np.random.rand(x.shape[0]) < 0.8
#                 temp_parameters = loc_param+ [_]
#                 simple_lr.fit(x[temp_parameters][temp_train_set], y[temp_train_set])
#                 linear_3rdround_mse = mse(simple_lr.predict(x[temp_parameters][~temp_train_set]), y[~temp_train_set])
#                 if linear_3rdround_mse < resultMSE:
#                     print 'a better MSE of', linear_3rdround_mse, 'with parameters set', temp_parameters
#                     resultMSE = linear_3rdround_mse
#                     loc_param = temp_parameters
#                     training_set_result.append(temp_train_set)



In [325]:
add_new_parameters(rna_train, ic50_train[2], new_parameters, 300, 0.8)

NameError: name 'new_parameters' is not defined

In [326]:
thirdroundselection = [7595, 8995, 9415, 14674, 18874, 1317, 4530, 9684, 2763]

In [ ]:
add_new_parameters(rna_train, ic50_train[2], thirdroundselection, 400, 0.8)

In [ ]:
fourthroundselection = [7595, 8995, 9415, 14674, 18874, 1317, 4530, 9684, 2763, 7255, 2593, 10260, 3232, 15405, 2802]

In [ ]:
add_new_parameters(rna_train, ic50_train[2], fourthroundselection, 300, 0.8, 1)

In [8]:
fifthroundselection = [7595, 8995, 9415, 14674, 18874, 1317, 4530, 9684, 2763, 7255, 2593, 10260, 3232, 15405, 2802, 11656, 16883]

In [65]:
len(fifthroundselection)

17

In [ ]:
lassocv = lassoCV()

In [ ]:
lassocv.fit(rna_train[fifthroundselection], ic50_train[2])

In [ ]:
lassocv.alpha_

In [ ]:
lassocv.coef_

In [ ]:
fifthroundlasso = []
for _ in range(len(fifthroundselection)):
    if lassocv.coef_[_] != 0:
        fifthroundlasso.append(fifthroundselection[_])

In [69]:
msetest(rna_train, ic50_train, fifthroundlasso)

NameError: name 'fifthroundlasso' is not defined

In [152]:
#compare to the 4 parameter model in the 1st round
#very close
msetest(rna_train, ic50_train, ['7595', '8995', '9415', '14674'])

KeyError: "['7595' '8995' '9415' '14674'] not in index"

In [ ]:
firstsubmission = lr()
firstsubmission.fit(rna_train[fifthroundlasso], ic50_train[2])

In [ ]:
ic50['1stroundpredict'] = firstsubmission.predict(rna[fifthroundlasso])

In [10]:
whole_feature = rna_train.join(cnv_train, rsuffix =  'cnv')

In [11]:
wholelasso = MultiTaskLassoCV()

In [12]:
wholelasso.fit(whole_feature, ic50_train)

MultiTaskLassoCV(alphas=None, copy_X=True, cv=None, eps=0.001,
         fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
         normalize=False, random_state=None, selection='cyclic',
         tol=0.0001, verbose=False)

In [13]:
len(wholelasso.coef_)

3

In [75]:
whole_parameters = [[], [], []]
for i in range(3):
    for _ in range(whole_feature.shape[1]):
        if wholelasso.coef_[i][_] != 0:
            whole_parameters[i].append(list(whole_feature)[_])
print whole_parameters

[['7595', '8995', '9415', '14674'], ['7595', '8995', '9415', '14674'], ['7595', '8995', '9415', '14674']]


In [25]:
add_new_parameters(whole_feature, ic50_train[2], ['7595', '8995', '9415', '14674'], 300, 0.8)

1


['7595', '8995', '9415', '14674']

In [9]:
#test the average cv MSE of the 7 parameters
def msetest(x, y, loc_param, split_ratio = 0.8, maxiter = 1000):
    simplelr = lr()
    mselist = []
    for _ in range(maxiter):
        temp_train_set = np.random.rand(x.shape[0]) < 0.8
        simplelr.fit(x[loc_param][temp_train_set], y[temp_train_set])
        mselist.append(mse(simplelr.predict(x[loc_param][~temp_train_set]), y[~temp_train_set]))
    return np.average(mselist)

In [10]:
main_feature = rna_train[[7595, 8995, 9415, 14674]]

In [11]:
main_feature

7595     8995     9415     14674
line_3    4.18405  13.4467  10.5276  12.6493
line_5    11.8301  6.80441  11.5342   14.033
line_7     9.7737   11.259  11.5608  13.5098
line_11   4.23034  14.2431  11.5918  11.7718
line_13   4.04256  4.02577  11.8529   8.5026
line_14   9.65427  12.3732  10.9056  11.8621
line_15   3.98276  5.96925  10.9961  13.2507
line_17   7.60546  14.2387   11.957  7.22757
line_18   8.32397  14.3003  13.3441   13.168
line_20   8.57541  4.18844  10.0915  10.5934
line_23    4.2213  3.65381  6.96088  6.62029
line_24   3.85645  4.06841  6.48536  10.4795
line_26   4.19607  13.2162  8.97061  13.6573
line_28   7.43582  14.1682  11.1208  9.34288
line_29    4.2031  3.60331  6.60509  10.4967
line_30   4.80219  12.0572   5.2954  8.54454
line_31   4.12283  13.0717  12.2875  13.5039
line_32   4.09032    13.26  11.5375  12.0606
line_33   11.3221  7.91627   11.601  10.6892
line_34   4.08152  9.43487  10.8814  11.5933
line_35   4.90479  14.1784  12.9975  13.1783
line_36   3.87874  13.3273  3.99162  5.40662
line_37    7.4983  12.9749   9.1003  5.62984
line_38   4.00805   13.261  11.9658  6.63086
line_39   9.47409  11.9928  10.6593  12.6801
line_40   12.3837  11.8972  12.2742  13.0356
line_42   4.68051  4.30491  11.9605  12.0524
line_43   9.11709  4.06792  10.8724  13.3964
line_44   9.05835  13.2986   11.576  9.48395
line_45   6.27221  13.0477  10.1877  13.9948
...           ...      ...      ...      ...
line_117   3.8799  3.49363  5.93118  11.5348
line_118  4.25617  3.86008  10.5928  12.9095
line_119  8.79671  5.32754  4.25373  4.61838
line_120  6.00285  9.21439  4.21161   4.7266
line_121  4.25822  13.1052   11.051  12.3212
line_122  9.19606  9.19732  12.5801  10.8424
line_123  4.88149   4.1413  12.6878  13.0668
line_124  8.77795  7.89873  10.1367  13.8406
line_125  7.96962  12.4529  10.4221  12.3538
line_126  5.45618    13.82  11.5187  6.65589
line_127  10.5439  13.9834  11.7215  13.4258
line_128  8.65747  13.4064  11.4128  12.1769
line_129  5.05074  13.7242  11.6385  11.2996
line_130  8.53496  13.6143  11.9775  12.3805
line_131  7.28202  13.4389  10.6998  11.1564
line_132  11.7525  9.90262  3.84744  9.44849
line_133  8.64562  4.36891  9.14777  11.4229
line_137  4.15591  4.87805  4.15005  5.52348
line_138  4.11144  4.40801  4.43178   6.9757
line_139   4.6195  10.4931  4.26221  4.67007
line_140  4.33173  4.00062  4.08522  4.57071
line_141  4.60224   4.2969  3.80255  4.62434
line_142  3.97325  3.74152  4.06185  4.75199
line_143  4.32739  3.73301  4.37092  4.76473
line_144  4.22041  3.93712   4.3023  9.59982
line_145  4.12255  4.02851  3.99041  4.82659
line_146  4.27136  4.29606  4.21996  4.60934
line_147  4.45602  3.95079  4.02806  6.40092
line_148  4.50854  3.77959  4.10854  4.93504
line_149  5.90325    3.809  4.33871  4.70499

[129 rows x 4 columns]

In [86]:
# ls24 = lr()
# ls40 = lr()
ls72 = lr()

In [87]:
# ls24.fit(main_feature, ic50_train[2])
# ls40.fit(main_feature, ic50_train[3])
ls72.fit(main_feature, ic50_train[4])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [174]:
mse(ls24.predict(main_feature), ic50_train[2])

879.72853989643409

In [92]:
fifthroundselection = [7595, 8995, 9415, 14674, 18874, 1317, 4530, 9684, 2763, 7255, 2593, 10260, 3232, 15405, 2802, 11656, 16883]

In [110]:
manyvariablelr = lr()

In [134]:
manyvariablelr.fit(rna_train[[7595, 8995, 9415, 14674]], ic50_train[2])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [216]:
msetest(rna_train, ic50_train[2], [7595, 8995, 9415, 14674])

972.77625648241076

In [223]:
msetest(rna_train, ic50_train[2], [7595, 8995, 14674, 1317, 9684, 7255, 10260, 3232, 11656, 16883])

868.53937025940229

In [260]:
best24 = lr()

In [261]:
best24.fit(rna_train[[7595, 8995, 14674, 1317, 9684, 7255, 10260, 3232, 11656, 16883]], ic50_train[2])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [262]:
ic50_test['24predict'] = best24.predict(rna_test[[7595, 8995, 14674, 1317, 9684, 7255, 10260, 3232, 11656, 16883]])

In [232]:
msetest(rna_train, ic50_train[3], [7595, 8995, 9415, 14674])

386.52385725857397

In [247]:
msetest(rna_train, ic50_train[3], [7595, 8995, 9415, 14674, 1317, 7255])

382.6857904823695

In [264]:
best40 = lr()
best40.fit(rna_train[[7595, 8995, 9415, 14674, 1317, 7255]], ic50_train[3])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [266]:
ic50_test['40predict'] = best40.predict(rna_test[[7595, 8995, 9415, 14674, 1317, 7255]])

In [359]:
add_new_parameters(rna_train[fifthroundselection], ic50_train[4], [7595, 8995, 9415, 14674], 0.2)

1


[7595, 8995, 9415, 14674, 2593, 2802, 11656]

In [12]:
best72 = lr()
best72.fit(rna_train[[7595, 8995, 14674, 1317, 9684, 7255, 10260, 3232, 11656, 16883]], ic50_train[4])
ic50_test['IC50_72hrs'] = best72.predict(rna_test[[7595, 8995, 14674, 1317, 9684, 7255, 10260, 3232, 11656, 16883]])

In [346]:
ic50_test.__delitem__('72predict')

In [348]:
ic50_test.to_csv('2ndresult')

In [67]:
msetest(rna_train, ic50_train[4], [7595, 8995, 9415, 14674])

0.95191314360091817

In [66]:
msetest(rna_train, ic50_train[4], [7595, 8995, 9415, 14674, 2593, 2802, 11656])

0.88414402372224687

In [14]:
thirdlr = lr()
thirdlr.fit(rna_train[[7595, 8995, 9415, 14674, 2593, 2802, 11656]], ic50_train[4])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
ic50_test['IC50_72hrs'] = thirdlr.predict(rna_test[[7595, 8995, 9415, 14674, 2593, 2802, 11656]])

In [16]:
ic50_test.to_csv('third.csv')

In [36]:
def find_best_training_set(x, y, loc_param, split_ratio = 0.8, maxiter = 10000):
    simplelr = lr()
    bestmse = 1
    train_set_list = []
    for _ in range(maxiter):
        temp_train_set = np.random.rand(x.shape[0]) < 0.8
        simplelr.fit(x[loc_param][temp_train_set], y[temp_train_set])
        a = mse(simplelr.predict(x[loc_param][~temp_train_set]), ic50_train[4][~temp_train_set])
        if a < bestmse:
            bestmse = a
            train_set_list.append(temp_train_set)
    print bestmse
    return train_set_list

In [37]:
setstrain = find_best_training_set(rna_train, ic50_train[4], [7595, 8995, 9415, 14674, 2593, 2802, 11656])

0.0893273526815


In [38]:
len(setstrain)

11

In [39]:
best_found_train = setstrain[-1]

In [60]:
lr72ic50 = lr()
lr72ic50.fit(rna_train[[7595, 8995, 9415, 14674, 2593, 2802, 11656]][best_found_train], ic50_train[4][best_found_train])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [61]:
mse(lr72ic50.predict(rna_train[[7595, 8995, 9415, 14674, 2593, 2802, 11656]]), ic50_train[4])

0.77547570890420303

In [62]:
a = lr72ic50.predict(rna_train[[7595, 8995, 9415, 14674, 2593, 2802, 11656]])

In [52]:
b = ic50_train[4]

In [63]:
ic50_test['IC50_72hrs'] = lr72ic50.predict(rna_test[[7595, 8995, 9415, 14674, 2593, 2802, 11656]])

In [64]:
ic50_test.to_csv('fifth.csv')